FastAPI 支持创建含子依赖项的依赖项。

并且，可以按需声明任意深度的子依赖项嵌套层级。

FastAPI 负责处理解析不同深度的子依赖项。

In [1]:
import uvicorn
from fastapi import Depends, FastAPI
app = FastAPI()

def query_extractor(q: str | None = None):
    return q

def query_or_cookie_extractor(
    q: str = Depends(query_extractor),
    last_query: str | None = "last query",
):
    if not q:
        return last_query
    return q

@app.get("/items/")
async def read_query(query_or_default: str = Depends(query_or_cookie_extractor)):
    return {"q_or_query": query_or_default}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [24572]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:64803 - "GET /items/?q=%E6%9D%8E%E5%9B%9B HTTP/1.1" 200 OK
INFO:     127.0.0.1:64825 - "GET /items/?last_query=%E5%BC%A0%E4%B8%89 HTTP/1.1" 200 OK
INFO:     127.0.0.1:64853 - "GET /items/?q=%E6%9D%8E%E5%9B%9B&last_query=%E5%BC%A0%E4%B8%89 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [24572]


In [ ]:
url = 'http://127.0.0.1:8009/items/?q=李四' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"q_or_query":"李四"}'

In [ ]:
url = 'http://127.0.0.1:8009/items/?last_query=张三' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"q_or_query":"张三"}'

In [ ]:
url = 'http://127.0.0.1:8009/items/?q=李四&last_query=张三' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"q_or_query":"李四"}'

注意，这里在路径操作函数中只声明了一个依赖项，即 query_or_cookie_extractor 。

但 FastAPI 必须先处理 query_extractor，以便在调用 query_or_cookie_extractor 时使用 query_extractor 返回的结果。

第一层依赖项query_extractor

第二层依赖项query_or_cookie_extractor

## 多次使用同一个依赖项

如果在同一个路径操作 多次声明了同一个依赖项，例如，多个依赖项共用一个子依赖项，FastAPI 在处理同一请求时，只调用一次该子依赖项。

FastAPI 不会为同一个请求多次调用同一个依赖项，而是把依赖项的返回值进行「缓存」，并把它传递给同一请求中所有需要使用该返回值的「依赖项」。

在高级使用场景中，如果不想使用「缓存」值，而是为需要在同一请求的每一步操作（多次）中都实际调用依赖项，可以把 Depends 的参数 use_cache 的值设置为 False :

In [ ]:
async def needy_dependency(fresh_value: str = Depends(get_value, use_cache=False)):
    return {"fresh_value": fresh_value}